In [13]:
import torch
import pandas as pd
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score,confusion_matrix
from imblearn.combine import SMOTETomek
overSampler=SMOTETomek()

from sklearn.model_selection import train_test_split


import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

In [14]:
data=pd.read_parquet('/home/hashmi/Files/DataFolder/ParticleBoxData/DownstreamData.parquet')

In [15]:
data=data[data['track_p']<100e3]
data.track_nLHCbIDs=data.track_nLHCbIDs.astype('int')


sample=data.sample(frac=0.01,random_state=46)

x=sample.drop(['Downstream',],axis=1)
y=sample['Downstream'].map({True:1,False:0})

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

x=ss.fit_transform(x)


In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,shuffle=True,stratify=y,random_state=29,test_size=0.3)

x_train,y_train=overSampler.fit_resample(X=x_train,y=y_train)

In [17]:
class DataFun(Dataset):
    def __init__(self,x_data,y_data):
        self.x_data=x_data
        self.y_data=y_data
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return len(self.x_data)

In [18]:
train_data=DataFun(torch.FloatTensor(x_train),torch.FloatTensor(y_train))

In [19]:
train_loader=DataLoader(train_data,batch_size=4,shuffle=True)

In [52]:
# class Network(nn.Module):
#     def __init__(self,input_num,output_num):
#         super(Network,self).__init__()
#         self.layer1=nn.Linear(input_num,15)
#         self.relu1=nn.ReLU()
#         self.layer2=nn.Linear(15,output_num)
#         self.sigmoid=nn.Sigmoid()

#     def forward(self,x):
#         output = self.layer1(x)
#         output = self.relu1(output)
#         output = self.layer2(output)
#         output = self.sigmoid(output)
#         return output

# model = Network(12,1)

In [54]:
class Network(nn.Module):
    def __init__(self,input_num,output_num):
        super(Network,self).__init__()
        self.layer1=nn.Linear(input_num,256)
        nn.init.kaiming_uniform_(self.layer1.weight,nonlinearity='relu')
        self.layer2=nn.Linear(256,128)
        nn.init.kaiming_uniform_(self.layer2.weight,nonlinearity='relu')
        self.layer3=nn.Linear(128,64)
        nn.init.kaiming_uniform_(self.layer3.weight,nonlinearity='relu')
        self.out=nn.Linear(64,output_num)
        nn.init.xavier_uniform_(self.out.weight)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(256)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)


    def forward(self,x):
        output = self.layer1(x)
        output = self.relu(output)
        output = self.batchnorm1(output)
        output = self.layer2(output)
        output = self.relu(output)
        output = self.batchnorm2(output)
        output = self.layer3(output)
        output = self.relu(output)
        output = self.batchnorm3(output)
        output = self.dropout(output)
        output = self.out(output)
        return output

model = Network(12,1)

In [55]:
epoch = 10
lr=0.01

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCEWithLogitsLoss()

# optimizer = torch.optim.Adam(params=model.parameters())
optimizer = torch.optim.SGD(params=model.parameters(),lr=lr)

In [56]:
# for epoch in range(1,epoch+1):

#     epoch_loss = 0

#     for x_batch,y_batch in train_loader:
#         optimizer.zero_grad()
#         y_predict = model(x_batch).detach()
#         loss = criterion(y_batch.unsqueeze(1),y_predict)
#         loss.backward()
#         optimizer.step()
#         # epoch_loss+=loss.item()
#         # print(y_batch.unsqueeze(1))
#         # print(y_predict.unsqueeze(1))
#         # print(loss)
#         break
#     break
#     # print(epoch,epoch_loss)

In [57]:
for epoch in range(1,10):

    epoch_loss = 0

    for x_batch,y_batch in train_loader:
        out=model(x_batch)
        loss=criterion(out,y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss+=loss.item()

    print(epoch,epoch_loss)

1 16695.517527014017
2 16470.48873566091
3 16345.794368833303
4 16262.607172742486
5 16206.313289627433
6 16128.49496717751
7 16108.285170122981
8 16057.281342700124
9 16036.96709547937


In [ ]:
1 16695.517527014017
2 16470.48873566091
3 16345.794368833303
4 16262.607172742486
5 16206.313289627433
6 16128.49496717751
7 16108.285170122981
8 16057.281342700124
9 16036.96709547937